In [1]:
!pip install -q espnet==0.10.6 pyopenjtalk==0.2 pypinyin==0.44.0 parallel_wavegan==0.5.4 gdown==4.4.0 espnet_model_zoo
!pip install typeguard==2.7.0

In [1]:
import torch
import os
import matplotlib.pyplot as plt
import numpy as np

import torchaudio
from torchaudio.datasets.librispeech import LIBRISPEECH
import torchaudio.transforms as transforms

In [2]:
PATH="/home/ubuntu/test/Synthetic ASR- IDL Project/"
COMPLETE_DATASET_PATH = PATH + "/librispeech"

if not os.path.exists(COMPLETE_DATASET_PATH):
  os.mkdir(COMPLETE_DATASET_PATH)

COMPLETE_DATASET = torchaudio.datasets.LIBRISPEECH(COMPLETE_DATASET_PATH, url='train-clean-100', download=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.95G/5.95G [07:02<00:00, 15.1MB/s]


In [3]:
#@title English multi-speaker pretrained model { run: "auto" }
lang = 'English'
tag = 'kan-bayashi/vctk_multi_spk_vits' #@param ["kan-bayashi/vctk_gst_tacotron2", "kan-bayashi/vctk_gst_transformer", "kan-bayashi/vctk_xvector_tacotron2", "kan-bayashi/vctk_xvector_transformer", "kan-bayashi/vctk_xvector_conformer_fastspeech2", "kan-bayashi/vctk_gst+xvector_tacotron2", "kan-bayashi/vctk_gst+xvector_transformer", "kan-bayashi/vctk_gst+xvector_conformer_fastspeech2", "kan-bayashi/vctk_multi_spk_vits", "kan-bayashi/vctk_full_band_multi_spk_vits", "kan-bayashi/libritts_xvector_transformer", "kan-bayashi/libritts_xvector_conformer_fastspeech2", "kan-bayashi/libritts_gst+xvector_transformer", "kan-bayashi/libritts_gst+xvector_conformer_fastspeech2", "kan-bayashi/libritts_xvector_vits"] {type:"string"}
vocoder_tag = "none" #@param ["none", "parallel_wavegan/vctk_parallel_wavegan.v1.long", "parallel_wavegan/vctk_multi_band_melgan.v2", "parallel_wavegan/vctk_style_melgan.v1", "parallel_wavegan/vctk_hifigan.v1", "parallel_wavegan/libritts_parallel_wavegan.v1.long", "parallel_wavegan/libritts_multi_band_melgan.v2", "parallel_wavegan/libritts_hifigan.v1", "parallel_wavegan/libritts_style_melgan.v1"] {type:"string"}

In [20]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    device="cuda",
    # device="cpu",
    speed_control_alpha=0.8,
    noise_scale=0.2,
    noise_scale_dur=0.2,
    length_scale= 
)

In [21]:
import glob
import os
import numpy as np
import kaldiio

# Get model directory path
from espnet_model_zoo.downloader import ModelDownloader
d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(tag)["train_config"])

def get_random_speaker():
  sids = None
  if text2speech.use_sids:
      spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
      with open(spk2sid) as f:
          lines = [line.strip() for line in f.readlines()]
      sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}
      
      # randomly select speaker
      sids = np.array(np.random.randint(1, len(sid2spk)))
      spk = sid2spk[int(sids)]
      return sids

In [ ]:
dataset = COMPLETE_DATASET

tts_model = text2speech

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, (waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id) in enumerate(dataset):

    if i % 500 == 0:
        print(i)
    transcript_str = dataset[i][2]

    wav = text2speech(transcript_str, sids=get_random_speaker())["wav"]
    wav = wav.squeeze().cpu().numpy()
    wav = np.expand_dims(wav, axis=0)

    filename = f"sample_{i}"
    torchaudio.save('/home/ubuntu/test/Synthetic ASR- IDL Project/new/Multi-Speaker/'+filename + '.wav',torch.from_numpy(wav) , sample_rate=16000)
    with open('/home/ubuntu/test/Synthetic ASR- IDL Project/new/multi-speaker-trans/sample_'+ str(i) + '.txt', 'w') as f:
        f.write(transcript_str)
